In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
final_dataset=df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['Current_Year']=2020
final_dataset['no_year']=final_dataset['Current_Year']-final_dataset['Year']
final_dataset.drop(['Year', 'Current_Year'], axis=1, inplace=True)

In [ ]:
final_dataset.head()

## Converting categorical features to one hot encoded

In [ ]:
final_dataset=pd.get_dummies(final_dataset, drop_first=True)

In [ ]:
final_dataset.head()

## Correlation Matrix

In [ ]:
corr_matrix = final_dataset.corr()
corr_matrix

## Plot Visualizations

In [ ]:
import seaborn as sns
sns.pairplot(final_dataset)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
corrmat=final_dataset.corr()
top_corr_features=corrmat.index
plt.figure(figsize=(20,20))
# plot heat map
g=sns.heatmap(final_dataset[top_corr_features].corr(), annot=True, cmap="RdYlGn")

## Train-Test Splitting

In [ ]:
from sklearn.model_selection import train_test_split
X = final_dataset.iloc[:,1:]
y = final_dataset.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.head()

In [ ]:
y.head()

## Feature Importance

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model=ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot graph of feature importances for better visualization 

feat_importances = pd.Series(model.feature_importances_, index=X.columns) 
feat_importances.nlargest(5).plot(kind='barh') 
plt.show()

## Model

In [ ]:
# Use the random grid to search for best hyperparameters 
# First create the base model to tune 

from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()

In [ ]:
## Hyperparameters
import numpy as np
n_estimators=[int(x) for x in np.linspace(start=100, stop=1200, num=12)]
print(n_estimators)

In [ ]:
#Randomized Search CV

#Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt'] 
#Haximum number of Levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num= 6)] 
#max_depth.append(None) #Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
#Minimum number of samples required at each teaf node
min_samples_leaf = [1, 2, 5, 10]
                

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Create the random_grid 

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features, 
               'max_depth': max_depth,
               'min_samples_split': min_samples_split, 
               'min_samples_leaf': min_samples_leaf}

print (random_grid)

In [ ]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, scoring='neg_mean_squared_error', n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=1)

In [ ]:
rf_random.fit(X_train, y_train)

In [ ]:
y_pred=rf_random.predict(X_test)
y_pred

## Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error
import math
print(mean_squared_error(y_test, y_pred))
print(math.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
sns.distplot(y_test-y_pred)

In [ ]:
plt.scatter(y_test, y_pred)

## R-Square and Adjusted R-Square

In [ ]:
import statsmodels.api as sm
X_addC = sm.add_constant(X_test)
result = sm.OLS(y_pred, X_addC).fit()
print(result.rsquared, result.rsquared_adj)